# Database examples

## Premise

Let's assume we want to upload a newly created isotherm in the internal database. This isotherm
is measured on the novel adsorbent *Carbon X1*, with nitrogen at 77 K. The sample itself is 
synthesised locally, by Dr. Alice and has then undergone a chemical activation process. The experiment is performed on a homemade apparatus by Dr. Bob.

## Imports

First we need to do the required imports

In [1]:
import pygaps

## Adsorbate

The internal database already contains nitrogen as an adsorbate therefore, there's no need to
worry about the adsorbate for now.

## Material

Since no samples are present in the internal database, we must first upload the sample object.
We create a Material class with the required values and try to upload it.

In [2]:
novel_material = pygaps.Material(
            name = 'Carbon',
            batch = 'X1',

            contact = 'Alice',
            source = 'local',
            treatment = 'etched'
            )

try:
    pygaps.material_to_db(pygaps.DATABASE, novel_material)
except Exception as e:
    print(e)

Error



We get a foreign key error, since the constraints do not exist in the database yet.
Before we can upload the sample, we need to have the values of the foreign keys in place:

- the contact, as Alice
- the source, as local
- the type of adsorbent, as novel
- the sample property we have added

We can do this by passing to the respective functions dictionaries with the required values.

In [3]:
pygaps.material_to_db_property_type(pygaps.DATABASE, { 'type' : 'contact'})
pygaps.material_to_db_property_type(pygaps.DATABASE, { 'type' : 'source'})
pygaps.material_to_db_property_type(pygaps.DATABASE, { 'type' : 'treatment'})

Material properties type uploaded contact
Material properties type uploaded source
Material properties type uploaded treatment


Now that all of the constraints are in place let's try the sample upload again.

In [4]:
pygaps.material_to_db(pygaps.DATABASE, novel_material)

Material uploaded Carbon X1


## Isotherm

Now it's time to upload the isotherm itself.
We happen to have the isotherm conveniently stored as a json file, so we load it into memory and we attempt to upload it.

In [5]:
from pathlib import Path
json_path = Path.cwd() / 'data' / 'database' / 'Carbon X1.json'
with open(json_path) as text_file:
    isotherm = pygaps.isotherm_from_json(text_file.read())

try:
    pygaps.isotherm_to_db(pygaps.DATABASE, isotherm)
except Exception as e:
    print(e)

Error



As before, we need to first make sure that the constraints are satisfied before the isotherm can be uploaded. On the isotherm side, we have the following constraints:

- the adsorbate which was used in the experiment, which is already in the database
- the machine which was used to measure the data
- the user who measured it
- the experiment type, in this case physisorption
- the experiment data types, which in this case are just the columns in the dataframe: pressure and loading.


Again, we pass to the respective functions dictionaries with the required values.

In [6]:
pygaps.isotherm_to_db_type(pygaps.DATABASE, {
                                 'type' : 'physisorption',
                                 'description' : 'physisorption at 77 k'
                             })
pygaps.isotherm_to_db_data_type(pygaps.DATABASE, {'type' : isotherm.loading_key})
pygaps.isotherm_to_db_data_type(pygaps.DATABASE, {'type' : isotherm.pressure_key})
pygaps.isotherm_to_db_property_type(pygaps.DATABASE, {'type':'activation_temperature'})
pygaps.isotherm_to_db_property_type(pygaps.DATABASE, {'type':'machine'})
pygaps.isotherm_to_db_property_type(pygaps.DATABASE, {'type':'user'})
pygaps.isotherm_to_db_property_type(pygaps.DATABASE, {'type':'lab'})

Experiment type uploaded physisorption
Experiment data type uploaded loading
Experiment data type uploaded pressure
Experiment property type uploaded activation_temperature
Experiment property type uploaded machine
Experiment property type uploaded user
Experiment property type uploaded lab


Now we can try the upload again.

In [7]:
pygaps.isotherm_to_db(pygaps.DATABASE, isotherm)

Success: Material: Carbon
Batch: X1
Adsorbate used: nitrogen
Isotherm temperature: 303.0K
iso_type: physisorption
Units: 
	Unit for loading: mmol/g
	Unit for pressure: bar
Other properties: 
	user: Bob
	machine: homemade1
	activation_temperature: 150.0
	lab: local



## Conclusion

While the instantiation process is initially lengthy, after the constraints are added to the database, the infrastructure is in place for easy upload of isotherms.

## Clean-up

If you run this iPython notebook with your own installation, run the following code to remove any residual values from the database.

In [8]:
pygaps.isotherm_delete_db(pygaps.DATABASE, isotherm)
pygaps.material_delete_db(pygaps.DATABASE, novel_material)
pygaps.material_delete_db_property_type(pygaps.DATABASE, 'treatment')
pygaps.material_delete_db_property_type(pygaps.DATABASE, 'source')
pygaps.material_delete_db_property_type(pygaps.DATABASE, 'contact')
pygaps.isotherm_delete_db_type(pygaps.DATABASE, 'physisorption')
pygaps.isotherm_delete_db_property_type(pygaps.DATABASE, 'activation_temperature')
pygaps.isotherm_delete_db_property_type(pygaps.DATABASE, 'machine')
pygaps.isotherm_delete_db_property_type(pygaps.DATABASE, 'user')
pygaps.isotherm_delete_db_property_type(pygaps.DATABASE, 'lab')
pygaps.isotherm_delete_db_data_type(pygaps.DATABASE, isotherm.loading_key)
pygaps.isotherm_delete_db_data_type(pygaps.DATABASE, isotherm.pressure_key)

Success: Material: Carbon
Batch: X1
Adsorbate used: nitrogen
Isotherm temperature: 303.0K
iso_type: physisorption
Units: 
	Unit for loading: mmol/g
	Unit for pressure: bar
Other properties: 
	user: Bob
	machine: homemade1
	activation_temperature: 150.0
	lab: local

Success Carbon X1
Success, deleted material property types treatment
Success, deleted material property types source
Success, deleted material property types contact
Success, deleted isotherm types physisorption
Success, deleted isotherm property types activation_temperature
Success, deleted isotherm property types machine
Success, deleted isotherm property types user
Success, deleted isotherm property types lab
Success, deleted isotherm data types loading
Success, deleted isotherm data types pressure
